In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# import os;
import pickle;
import numpy as np;
import re
from collections import OrderedDict
import pandas as pd

from keras.preprocessing.text import text_to_word_sequence;
from keras.models import Sequential,load_model;
from keras.utils.vis_utils import plot_model;


from keras.models import Model;
from keras.layers.embeddings import Embedding;
from keras.models import Sequential,load_model;
from keras.optimizers import rmsprop,adam,adagrad,SGD;
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau, CSVLogger;
from keras.preprocessing.text import text_to_word_sequence,one_hot,Tokenizer;
from keras.layers import Input,Dense,merge,Dropout,BatchNormalization,Activation,Conv1D;


from datetime import datetime

# setting current working directory

DIR='/home/ivan/Dropbox/Projects/OldLang/code/ByteNet-Keras-master'
WKDIR = DIR
data_path = '/home/ivan/Dropbox/Projects/OldLang/data/ancient/' + 'processed_v6_maxlen205.txt' #!!!

batch_size=64
#N=15040
maxlen=205;

Using TensorFlow backend.


In [3]:
model_path = '/home/ivan/Dropbox/Projects/OldLang/code/ByteNet-Keras-master/models/' +'conv1d_54'#'conv1d_30'

In [4]:
def load_dataset(batch_size, data_path, N=None):
    
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    
    if N is not None:
        lines = lines[:N]

    anc_list = []
    rus_list = []
    for line in lines[: len(lines) - 1]:
        anc, rus = line.split('\t')
        anc_list += [anc]
        rus_list += [rus]
    
    French = rus_list
    English = anc_list
    English = [i + "\n" for i in English];# add ending signal at the sequence end
    
    if batch_size is not None:
        while 1:
            if len(English) % batch_size != 0:
                del English[-1];
                del French[-1];
            else:
                break;
    return French,English;


#lan1, lan2 = load_dataset(batch_size=64, data_path=data_path, N=None) #!!!

In [5]:

def build_vacabulary(French,English):
    all_eng_words = [];
    all_french_words = [];
    for i in np.arange(0, len(French)):
        all_eng_words.append(English[i]);
        all_french_words.append(French[i]);
    tokeng = Tokenizer(char_level=True);
    tokeng.fit_on_texts(all_eng_words);
    eng_index = tokeng.word_index;  # build character to index dictionary
    index_eng = dict((eng_index[i], i) for i in eng_index);
    tokita = Tokenizer(char_level=True);
    tokita.fit_on_texts(all_french_words);
    french_index = tokita.word_index;  # build character to index dictionary
    index_french = dict((french_index[i], i) for i in french_index);
    return (eng_index,french_index,index_eng,index_french);

In [6]:
French,English=load_dataset(batch_size=batch_size, data_path=data_path, N=None);
eng_index, french_index, index_eng, index_french=build_vacabulary(French,English);

In [7]:
len(French)

15040

In [8]:
eng_index, french_index, index_eng, index_french

({' ': 1,
  'о': 2,
  'и': 3,
  'е': 4,
  'а': 5,
  'с': 6,
  'т': 7,
  'н': 8,
  'в': 9,
  'р': 10,
  'м': 11,
  'л': 12,
  'д': 13,
  'у': 14,
  'п': 15,
  'к': 16,
  'я': 17,
  'ъ': 18,
  'г': 19,
  'ѣ': 20,
  'б': 21,
  'ь': 22,
  'ж': 23,
  'ы': 24,
  'з': 25,
  '\n': 26,
  'ш': 27,
  'х': 28,
  'ч': 29,
  'ю': 30,
  'й': 31,
  'ц': 32,
  'щ': 33,
  'ф': 34,
  '0': 35,
  '6': 36,
  '1': 37,
  '4': 38,
  '5': 39,
  '3': 40,
  '7': 41,
  '2': 42,
  '8': 43,
  '9': 44,
  'ѝ': 45,
  'э': 46,
  'і': 47,
  'ё': 48,
  'à': 49,
  'c': 50,
  'i': 51},
 {' ': 1,
  'о': 2,
  'и': 3,
  'е': 4,
  'а': 5,
  'с': 6,
  'т': 7,
  'н': 8,
  'в': 9,
  'л': 10,
  'р': 11,
  'м': 12,
  'д': 13,
  'к': 14,
  'п': 15,
  'у': 16,
  'г': 17,
  'я': 18,
  'б': 19,
  'ы': 20,
  'ь': 21,
  'з': 22,
  'ч': 23,
  'й': 24,
  'ж': 25,
  'х': 26,
  'ш': 27,
  'ю': 28,
  'ц': 29,
  'щ': 30,
  'э': 31,
  'ф': 32,
  '6': 33,
  '4': 34,
  '7': 35,
  '5': 36,
  'ъ': 37,
  '1': 38,
  '0': 39,
  '3': 40,
  '2': 41,
  '9

In [9]:
model=load_model(model_path);

/home/ivan/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [10]:
def input_2_vec(input):
    source=input[0];target=input[1];
    source_vec=np.zeros((1,maxlen+1),dtype=np.uint16);
    target_vec=np.zeros((1,maxlen),dtype=np.uint16);
    for i,ele1 in enumerate(source):
        source_vec[0,i]=french_index[ele1];
    for j,ele2 in enumerate(target):
        target_vec[0,j]=eng_index[ele2];
    return (source_vec,target_vec);

def T(sentence):
    vec=input_2_vec(sentence);
    source_vec=vec[0];target_vec=vec[1];
    t0=np.zeros((1,1,500),dtype=np.uint8);
    predict=model.predict([source_vec,target_vec,t0])[0];
    predict_max=np.argmax(predict,axis=-1);
    answers=[index_eng[j+1] for j in predict_max];
    a="".join(answers);
    return a;

#plot_model(model, to_file="model.png", show_shapes=True)
def translate(french_sentence):
    french_sentence = preproc_test(french_sentence)
    process_english_sentence="";
    length=0;
    while 1:
        predicted_english_sentence=T([french_sentence,process_english_sentence]);
        process_english_sentence=predicted_english_sentence[:length+1];length+=1;
        if process_english_sentence[-1]=="\n":break;
        if length>=maxlen+1:break;
        #if length%10==0:print("{} completed".format(str(length*maxlen**-1)));
    return process_english_sentence.strip();


In [23]:
def preproc_test(line):
    
    replace_dict = OrderedDict({
                                    '<...>': '',
                                    '«': '', '»': '',
                                    '...': '',
                                    '!.': '.',
                                    '!': '.',
                                    '?.': '.',
                                    '?': '.',     
                                    ';': ',', #!!!
    })
    
    for k in replace_dict.keys():
        line = line.replace(k, replace_dict[k])
        
    
    line = re.sub(r"\[\d+\]", "", line)
    line = re.sub(r"\(\d+\)", "", line)
    line = re.sub(r" \.", ".", line)

    replace_dict_2 = OrderedDict({
        '.,': '.',
        '—': '',
        '“': '',
        '”': '',
        '’': '',
        '˝': '',
        '"': '',
        '´': '',
        '(': '', ')': '',
        '<': '', '>': '',
        '-': '', 
        '„': '',
        '̀': '',
        '́': '',
        '*': '',
        '…': '',  
        "'": '',
        '{': '',
        '}': '',
                                
    })
    
    for k in replace_dict_2.keys():
        line = line.replace(k, replace_dict_2[k])
                

    line = line.replace('\t', ' ')
    line = line.replace(',', '') #!!
    #line = line.replace('.', '') #!!!
    line = line.replace(':', '')
    
    line = line.replace('.', '')
    
    line = line.lower() #!!!
    
    return line

In [29]:
s = """И сказал, смеясь, Иван Васильевич
Твоему горю пособить постараюся
И пошли дары драгоценные
И сказал мне так тихим шепотом
Ох ты гой еси, царь Иван Васильевич!
Обманул тебя твой лукавый раб
И такое слово ему молвили
Когда сизый орел зовет голосом
А такой обиды не стерпеть душе"""

In [30]:
df = []

for x in s.split('\n'):
    y = translate(x)
    df += [[x, y]]
    
df = pd.DataFrame(df)
df.columns = ['Оригинал', "Сгенерировано"]   
df

,Оригинал,Сгенерировано
0,"И сказал, смеясь, Иван Васильевич",и рече князи сынъ иван василковъ великий
1,Твоему горю пособить постараюся,твоему господи поспѣшити да сподобиться
2,И пошли дары драгоценные,и поидоша дарования дождеся
3,И сказал мне так тихим шепотом,и глагола ми тако те позаемь
4,"Ох ты гой еси, царь Иван Васильевич!",охъ же сей царь и васильевича видѣх
5,Обманул тебя твой лукавый раб,помышляше братие твоя рабъ врус
6,И такое слово ему молвили,и тако словесе моляшеся
7,Когда сизый орел зовет голосом,и сиа судом отроча велми глаголаше
8,А такой обиды не стерпеть душе,такоже и обрѣте не достоит душю
